In [2]:
import numpy as np              
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import palettable              # 配色方案
from plotnine import *         # ggplot2的配色布局方案
%matplotlib inline

import scienceplots
plt.style.use('science')   
import warnings
warnings.filterwarnings('ignore')
plt.style.use('no-latex')
plt.rcParams['font.family'] = "Times New Roman"

In [3]:
heroes=pd.concat([pd.read_excel('mk-heroes-audit-5-times-and-killers (2).xlsx'),pd.read_excel('mk36-report (1).xlsx')])
heroes.reset_index(inplace=True)
heroes

,index,name,type,unitlevel,Episode,kills,time,deathMinutes,birthMinutes,killedBy
0,0,Agobard,Knight,6,Marble Kingdoms 10,126,2392.00,39.866666,0.016666,Amara
1,1,Caelia,Mage,4,Marble Kingdoms 10,51,3009.00,NaN,17.816666,NaN
2,2,Crispin,Fighter,5,Marble Kingdoms 10,48,3207.00,53.450000,19.733333,Labyorteaux
3,3,Wibert,Mage,4,Marble Kingdoms 10,46,3380.00,NaN,12.133333,NaN
4,4,Ferumald,Mage,5,Marble Kingdoms 10,43,1412.00,23.533333,13.866666,Captain Gojenola
...,...,...,...,...,...,...,...,...,...,...
4227,230,Obstagoon XII,Prince,0,Marble Kingdoms 36,1,394.56,6.566666,0.000000,Gojenola
4228,231,Tou II,Fighter,0,Marble Kingdoms 36,1,2273.91,37.883333,0.000000,Menacelafusion
4229,232,Artems III,Captain,0,Marble Kingdoms 36,1,3637.41,60.616666,58.083333,Joanwerd II
4230,233,Fenrir VIII,Captain,0,Marble Kingdoms 36,1,6189.06,103.150000,69.066666,NaN


In [4]:
# Correct way to extract last two characters from each value in the Episode column
heroes['Episode']=heroes['Episode'].str[-2:]
heroes['Episode']=heroes['Episode'].astype(int)
heroes.drop(['index'],axis=1)
heroes

,index,name,type,unitlevel,Episode,kills,time,deathMinutes,birthMinutes,killedBy
0,0,Agobard,Knight,6,10,126,2392.00,39.866666,0.016666,Amara
1,1,Caelia,Mage,4,10,51,3009.00,NaN,17.816666,NaN
2,2,Crispin,Fighter,5,10,48,3207.00,53.450000,19.733333,Labyorteaux
3,3,Wibert,Mage,4,10,46,3380.00,NaN,12.133333,NaN
4,4,Ferumald,Mage,5,10,43,1412.00,23.533333,13.866666,Captain Gojenola
...,...,...,...,...,...,...,...,...,...,...
4227,230,Obstagoon XII,Prince,0,36,1,394.56,6.566666,0.000000,Gojenola
4228,231,Tou II,Fighter,0,36,1,2273.91,37.883333,0.000000,Menacelafusion
4229,232,Artems III,Captain,0,36,1,3637.41,60.616666,58.083333,Joanwerd II
4230,233,Fenrir VIII,Captain,0,36,1,6189.06,103.150000,69.066666,NaN


In [5]:
heroes['deathSeconds']=(heroes['deathMinutes']*60).round().astype('Int64')
heroes['birthSeconds']=(heroes['birthMinutes']*60).round().astype('Int64')
heroes

,index,name,type,unitlevel,Episode,kills,time,deathMinutes,birthMinutes,killedBy,deathSeconds,birthSeconds
0,0,Agobard,Knight,6,10,126,2392.00,39.866666,0.016666,Amara,2392,1
1,1,Caelia,Mage,4,10,51,3009.00,NaN,17.816666,NaN,<NA>,1069
2,2,Crispin,Fighter,5,10,48,3207.00,53.450000,19.733333,Labyorteaux,3207,1184
3,3,Wibert,Mage,4,10,46,3380.00,NaN,12.133333,NaN,<NA>,728
4,4,Ferumald,Mage,5,10,43,1412.00,23.533333,13.866666,Captain Gojenola,1412,832
...,...,...,...,...,...,...,...,...,...,...,...,...
4227,230,Obstagoon XII,Prince,0,36,1,394.56,6.566666,0.000000,Gojenola,394,0
4228,231,Tou II,Fighter,0,36,1,2273.91,37.883333,0.000000,Menacelafusion,2273,0
4229,232,Artems III,Captain,0,36,1,3637.41,60.616666,58.083333,Joanwerd II,3637,3485
4230,233,Fenrir VIII,Captain,0,36,1,6189.06,103.150000,69.066666,NaN,6189,4144


In [6]:
# Function to convert seconds to "MM:SS" format (handles NaN)
def seconds_to_mmss(seconds_series):
    mmss = (
        seconds_series.dropna()  # Remove NaN to avoid errors
        .apply(lambda x: f"{int(x // 60)}:{int(x % 60):02d}")  # Format as MM:SS
    )
    return mmss.reindex(seconds_series.index)  # Reinsert NaN where original was NaN

# Apply to birth and death columns
heroes['birthTime'] = seconds_to_mmss(heroes['birthSeconds'])
heroes['deathTime'] = seconds_to_mmss(heroes['deathSeconds'])

# Display the result
heroes[['birthTime', 'deathTime']]

,birthTime,deathTime
0,0:01,39:52
1,17:49,NaN
2,19:44,53:27
3,12:08,NaN
4,13:52,23:32
...,...,...
4227,0:00,6:34
4228,0:00,37:53
4229,58:05,60:37
4230,69:04,103:09


In [7]:
# Function to format seconds as H:MM:SS (or MM:SS if H=0), omitting leading zeros
def format_hms(seconds_series):
    def single_value(x):
        if pd.isna(x):
            return np.nan
        x = int(x)
        h, remainder = divmod(x, 3600)
        m, s = divmod(remainder, 60)
        if h > 0:
            return f"{h}:{m:02d}:{s:02d}"  # H:MM:SS
        else:
            return f"{m}:{s:02d}"  # MM:SS (no hours)
    
    return seconds_series.apply(single_value)

# Apply formatting
heroes['birthTimeHMS'] = format_hms(heroes['birthSeconds'])
heroes['deathTimeHMS'] = format_hms(heroes['deathSeconds'])

# Show results
heroes[['birthSeconds', 'birthTimeHMS', 'deathSeconds', 'deathTimeHMS']]

,birthSeconds,birthTimeHMS,deathSeconds,deathTimeHMS
0,1,0:01,2392,39:52
1,1069,17:49,<NA>,NaN
2,1184,19:44,3207,53:27
3,728,12:08,<NA>,NaN
4,832,13:52,1412,23:32
...,...,...,...,...
4227,0,0:00,394,6:34
4228,0,0:00,2273,37:53
4229,3485,58:05,3637,1:00:37
4230,4144,1:09:04,6189,1:43:09


In [8]:
# Make a clean copy of the type column by removing Lvl suffixes
heroes['clean_type'] = heroes['type'].str.replace(r' Lvl\d+$', '', regex=True)

# Identify summoned heroes where clean_type matches name OR type exactly matches name
summoned_mask = (
    (heroes['clean_type'] == heroes['name']) | 
    (heroes['type'] == heroes['name'])
)

# Apply the filter
summoned_heroes = heroes[summoned_mask].copy()
normal_heroes = heroes[~summoned_mask].copy()

# Clean up (optional)
heroes.drop(columns=['clean_type'], inplace=True, errors='ignore')

# Verify results
print(f"Found {len(summoned_heroes)} summoned heroes")
summoned_heroes=summoned_heroes.drop(['type'],axis=1)
summoned_heroes.rename(columns={'clean_type':'type'},inplace=True) 
summoned_heroes=summoned_heroes[list(heroes.columns)]
summoned_heroes

Found 81 summoned heroes


,index,name,type,unitlevel,Episode,kills,time,deathMinutes,birthMinutes,killedBy,deathSeconds,birthSeconds,birthTime,deathTime,birthTimeHMS,deathTimeHMS
1572,1572,Pretox,Pretox,8,21,28,4390.00,73.166666,0.016666,Rosemunda,4390,1,0:01,73:10,0:01,1:13:10
1689,1689,Askgar,Askgar,9,22,178,4572.00,76.200000,32.933333,Ellioteaster,4572,1976,32:56,76:12,32:56,1:16:12
1692,1692,Donston,Donston,5,22,36,3295.00,54.916666,14.933333,Askgar,3295,896,14:56,54:55,14:56,54:55
1695,1695,Corncrusader,Corncrusader,5,22,26,3360.00,56.000000,9.483333,Noinfo,3360,569,9:29,56:00,9:29,56:00
1710,1710,Yassir,Yassir,4,22,9,1962.00,32.700000,9.450000,Arthur,1962,567,9:27,32:42,9:27,32:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4003,6,wangxiaotao,wangxiaotao,7,36,13,4712.73,78.533333,72.216666,Walpurgisnacht,4712,4333,72:13,78:32,1:12:13,1:18:32
4010,13,Princeoftara,Princeoftara,5,36,23,3150.66,52.500000,36.066666,Egan Birgen,3150,2164,36:04,52:30,36:04,52:30
4014,17,Ailmar,Ailmar,4,36,1,3906.06,65.100000,58.333333,Stanhall II,3906,3500,58:20,65:06,58:20,1:05:06
4060,63,Segaraund II,Segaraund II,2,36,5,6114.12,NaN,101.900000,NaN,<NA>,6114,101:54,NaN,1:41:54,NaN


In [9]:
list(heroes.columns)

['index',
 'name',
 'type',
 'unitlevel',
 'Episode',
 'kills',
 'time',
 'deathMinutes',
 'birthMinutes',
 'killedBy',
 'deathSeconds',
 'birthSeconds',
 'birthTime',
 'deathTime',
 'birthTimeHMS',
 'deathTimeHMS']

In [10]:
# Define title patterns (Captain/King/Prince) and MK threshold
titles_to_remove = r'^(Captain|King|Prince)\s+'
false_title_threshold = 20  # MK 20+ are false titles

# Clean names:
# - Remove titles ONLY for episodes BEFORE MK 20 (genuine titles)
# - Preserve original names for MK 20+ (false titles)
heroes['cleaned_name'] = heroes['name'].where(
    heroes['Episode'] >= false_title_threshold,  # Keep if MK 20+
    heroes['name'].str.replace(titles_to_remove, '', case=False, regex=True)
)

# Apply same logic to 'type' column if needed
heroes['cleaned_type'] = heroes['type'].where(
    heroes['Episode'] >= false_title_threshold,
    heroes['type'].str.replace(titles_to_remove, '', case=False, regex=True)
)

# Verify changes (compare original vs cleaned names)
heroes=heroes.drop(['name'],axis=1)
heroes.rename(columns={'cleaned_name':'name'},inplace=True) 
heroes=heroes[list(summoned_heroes.columns)]
heroes

,index,name,type,unitlevel,Episode,kills,time,deathMinutes,birthMinutes,killedBy,deathSeconds,birthSeconds,birthTime,deathTime,birthTimeHMS,deathTimeHMS
0,0,Agobard,Knight,6,10,126,2392.00,39.866666,0.016666,Amara,2392,1,0:01,39:52,0:01,39:52
1,1,Caelia,Mage,4,10,51,3009.00,NaN,17.816666,NaN,<NA>,1069,17:49,NaN,17:49,NaN
2,2,Crispin,Fighter,5,10,48,3207.00,53.450000,19.733333,Labyorteaux,3207,1184,19:44,53:27,19:44,53:27
3,3,Wibert,Mage,4,10,46,3380.00,NaN,12.133333,NaN,<NA>,728,12:08,NaN,12:08,NaN
4,4,Ferumald,Mage,5,10,43,1412.00,23.533333,13.866666,Captain Gojenola,1412,832,13:52,23:32,13:52,23:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,230,Obstagoon XII,Prince,0,36,1,394.56,6.566666,0.000000,Gojenola,394,0,0:00,6:34,0:00,6:34
4228,231,Tou II,Fighter,0,36,1,2273.91,37.883333,0.000000,Menacelafusion,2273,0,0:00,37:53,0:00,37:53
4229,232,Artems III,Captain,0,36,1,3637.41,60.616666,58.083333,Joanwerd II,3637,3485,58:05,60:37,58:05,1:00:37
4230,233,Fenrir VIII,Captain,0,36,1,6189.06,103.150000,69.066666,NaN,6189,4144,69:04,103:09,1:09:04,1:43:09


In [28]:
kingdoms=pd.read_excel('MK Leaderboard.xlsx')
print(kingdoms)

    Kingdom MK 1 MK 2 MK 3 MK 4 MK 5 MK 6 MK 7 MK 8 MK 9  ... MK 27 MK 28  \
0       Red    2    -    2    4    0    4    3    2    2  ...     7     1   
1     Green    4    1    1    5    0    2    4    1    -  ...     4     6   
2      Blue    1    3    4    7    0    3    2    -    4  ...     8     3   
3    Yellow    3    4    3    3    2    1    1    -    1  ...     1     9   
4    Orange    -    -    -    2    -    -    -    4    -  ...     5     8   
5      Cyan    -    -    -    8    -    -    -    -    -  ...     3     2   
6    Purple    -    2    -    6    -    -    -    3    -  ...     6     7   
7   Magenta    -    -    -    1    -    -    -    -    -  ...     -     -   
8      Teal    -    -    -    -    -    -    -    -    3  ...     2     -   
9      Pink    -    -    -    -    -    -    -    -    -  ...     9     4   
10     Lime    -    -    -    -    -    -    -    -    -  ...     -     -   
11    Beige    -    -    -    -    -    -    -    -    -  ...     -     5   

In [14]:
import pywikibot as pwb
import webbrowser as wb
import requests
import re
import os

site = pwb.Site("en", "marble_kingdoms")  # (language_code, family)
site.login()

Logging in to marble_kingdoms:en as Louisa-Bot@Louisa-Bot


In [29]:
KINGDOM_COLORS = {
    'Red': '#FF0000',
    'Orange': '#FFA500',
    'Yellow': '#FFFF00',
    'Green': '#00C000',
    'Cyan': '#00FFFF',
    'Blue': '#0000FF',
    'Purple': '#8000FF',
    'Magenta': '#FF00FF',
    'Pink': '#FF80FF',
    'Lime': '#80FF60',
    'Teal': '#008080',
    'Beige': '#F5F5DC',
    'Gray': '#A9A9A9',
    'Darkgray': '#404040'
}

def format_cell(value, position, is_last=False):
    """Format cell with center alignment and appropriate styling"""
    base_style = 'text-align:center;'
    
    if value == '-':
        return f'style="{base_style}" data-sort-value="-1" | -'
    elif value == 0:
        return f'style="{base_style} background:gray; color:white;" data-sort-value="0" | 0'
    
    if position == 1:
        style = f'{base_style} background:gold; color:black;'
    elif position == 2:
        style = f'{base_style} background:silver; color:black;'
    elif position == 3:
        style = f'{base_style} background:#cd7f32; color:black;'  # bronze
    elif is_last:
        style = f'{base_style} background:red; color:white;'
    else:
        style = base_style
    
    return f'style="{style}" data-sort-value="{value}" | {value}'

def create_formatted_wikitable(df):
    """Create a formatted wikitable with each cell on its own line and proper linking"""
    # Determine last places for each MK
    last_places = {}
    for mk in df.columns[1:]:  # Skip 'Kingdom'
        valid_values = [x for x in df[mk] if isinstance(x, (int, float)) and x > 0]
        last_places[mk] = max(valid_values) if valid_values else None
    
    # Start table
    wiki_text = '{| class="wikitable sortable"\n'
    wiki_text += '|+\n'
    
    # Header row - each MK on its own line with link
    wiki_text += '! Kingdom\n'
    for mk in df.columns[1:]:
        wiki_text += f'! [[{mk}]]\n'
    
    # Data rows
    for _, row in df.iterrows():
        wiki_text += '|-\n'
        
        # Format kingdom name with color
        kingdom = row['Kingdom']
        color = KINGDOM_COLORS.get(kingdom, 'inherit')
        wiki_text += f'| style="color:{color}; font-weight:bold;" | {kingdom}\n'
        
        for mk in df.columns[1:]:
            value = row[mk]
            
            # Format cell
            if value == '-':
                cell = 'style="text-align:center;" data-sort-value="-1" | -'
            elif value == 0:
                cell = 'style="background:gray; color:white; text-align:center;" data-sort-value="0" | 0'
            else:
                # Determine styling
                if value == 1:
                    style = 'background:gold; color:black; text-align:center;'
                elif value == 2:
                    style = 'background:silver; color:black; text-align:center;'
                elif value == 3:
                    style = 'background:#cd7f32; color:black; text-align:center;'
                elif value == last_places.get(mk):
                    style = 'background:red; color:white; text-align:center;'
                else:
                    style = 'text-align:center;'
                
                cell = f'style="{style}" data-sort-value="{value}" | {value}'
            
            wiki_text += f'| {cell}\n'
    
    wiki_text += '|}\n'
    return wiki_text

def upload_to_fandom(table_text, page_title, summary):
    """Upload table to Fandom using pywikibot"""
    page = pwb.Page(site, page_title)
    page.text = table_text
    page.save(summary)

# Assuming your dataframe is called 'kingdoms'
# Example usage:
# kingdoms = pd.DataFrame({...})  # Your dataframe here

# Generate the wikitable
wikitable = create_formatted_wikitable(kingdoms)

# Upload to Fandom
page_title = "User:2007K2FeO4/Sandbox/Podium Details"
summary = "Updating leaderboard with formatted positions"
upload_to_fandom(wikitable, page_title, summary)

print("Table successfully uploaded to Fandom!")

Table successfully uploaded to Fandom!


Page [[User:2007K2FeO4/Sandbox/Podium Details]] saved
